In [1]:
# 导入所需的package
import seaborn as sns #用于画图
from bs4 import BeautifulSoup #用于爬取arxiv的数据
import re #用于正则表达式，匹配字符串的模式
import requests #用于网络连接，发送网络请求，使用域名获取对应信息
import json #读取数据，我们的数据为json格式的
import pandas as pd #数据处理，数据分析
import matplotlib.pyplot as plt #画图工具

******************
## 目录

**一、任务说明**


**二、数据集介绍**


**三、arxiv论文类别介绍**

**四、具体代码实现以及讲解**

* 4.1 导入package并读取原始数据
* 4.2 数据预处理
* 4.3 数据分析及可视化


## 正式学习内容
**************

### 一、任务说明

* 任务主题：论文数量统计，即统计2019年全年计算机各个方向论文数量；
* 任务内容：赛题的理解、使用 Pandas 读取数据并进行统计；
* 任务成果：学习 Pandas 的基础操作；
* 可参考的学习资料：[开源组织Datawhale joyful-pandas项目](https://github.com/datawhalechina/joyful-pandas)（刚从那边过来,非常硬的项目）

### 二、数据集介绍

* 数据集来源：[数据集链接](https://www.kaggle.com/Cornell-University/arxiv)；
  
    
* 数据集的格式如下：

    * id：arXiv ID，可用于访问论文；
    * submitter：论文提交者；
    * authors：论文作者；
    * title：论文标题；
    * comments：论文页数和图表等其他信息；
    * journal-ref：论文发表的期刊的信息；
    * doi：数字对象标识符，https://www.doi.org；
    * report-no：报告编号；
    * categories：论文在 arXiv 系统的所属类别或标签；
    * license：文章的许可证；
    * abstract：论文摘要；
    * versions：论文版本；
    * authors_parsed：作者的信息。
      

  
* 数据集实例



```json
"root":{
		"id":string"0704.0001"
		"submitter":string"Pavel Nadolsky"
		"authors":string"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan"
		"title":string"Calculation of prompt diphoton production cross sections at Tevatron and LHC energies"
		"comments":string"37 pages, 15 figures; published version"
		"journal-ref":string"Phys.Rev.D76:013009,2007"
		"doi":string"10.1103/PhysRevD.76.013009"
		"report-no":string"ANL-HEP-PR-07-12"
		"categories":string"hep-ph"
		"license":NULL
		"abstract":string"  A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events."
		"versions":[
				0:{
						"version":string"v1"
						"created":string"Mon, 2 Apr 2007 19:18:42 GMT"
					}
				1:{
						"version":string"v2"
						"created":string"Tue, 24 Jul 2007 20:10:27 GMT"
					}]
		"update_date":string"2008-11-26"
		"authors_parsed":[
				0:[
						0:string"Balázs"
						1:string"C."
						2:string""]
				1:[
						0:string"Berger"
						1:string"E. L."
						2:string""]
				2:[
						0:string"Nadolsky"
						1:string"P. M."
						2:string""]
				3:[
						0:string"Yuan"
						1:string"C. -P."
						2:string""]]
}

```


### 三、arxiv论文介绍

这里不展开了，可以看一下[原文档](https://github.com/datawhalechina/team-learning-data-mining/blob/master/AcademicTrends/Task1%20%E8%AE%BA%E6%96%87%E6%95%B0%E6%8D%AE%E7%BB%9F%E8%AE%A1.md)

### 四、具体代码实现

#### 4.1 导入package并读取原始数据

In [2]:
# 导入所需的package
import seaborn as sns #用于画图
from bs4 import BeautifulSoup #用于爬取arxiv的数据
import re #用于正则表达式，匹配字符串的模式
import requests #用于网络连接，发送网络请求，使用域名获取对应信息
import json #读取数据，我们的数据为json格式的
import pandas as pd #数据处理，数据分析
import matplotlib.pyplot as plt #画图工具

* 读入数据

In [ ]:
data  = [] #初始化

#使用with语句优势：1.自动关闭文件句柄；2.自动显示（处理）文件读取数据异常
with open("D:/z_personal_file/data/arxiv-metadata-oai-snapshot.json", 'r') as f: 
    for line in f: 
        data.append(json.loads(line))
        
data = pd.DataFrame(data) #将list变为dataframe格式，方便使用pandas进行分析
data.shape #显示数据大小

In [ ]:
data.head()

* 数据预处理

In [5]:
# 查看论文种类信息

data["categories"].describe()

count      1796911
unique       62055
top       astro-ph
freq         86914
Name: categories, dtype: object

* 查看本数据集种出现多少种独立的数据集

In [8]:
categories = [x.split(' ') for x in data["categories"]] # 先分割
categories

[['hep-ph'],
 ['math.CO', 'cs.CG'],
 ['physics.gen-ph'],
 ['math.CO'],
 ['math.CA', 'math.FA'],
 ['cond-mat.mes-hall'],
 ['gr-qc'],
 ['cond-mat.mtrl-sci'],
 ['astro-ph'],
 ['math.CO'],
 ['math.NT', 'math.AG'],
 ['math.NT'],
 ['math.NT'],
 ['math.CA', 'math.AT'],
 ['hep-th'],
 ['hep-ph'],
 ['astro-ph'],
 ['hep-th'],
 ['math.PR', 'math.AG'],
 ['hep-ex'],
 ['nlin.PS', 'physics.chem-ph', 'q-bio.MN'],
 ['math.NA'],
 ['astro-ph'],
 ['nlin.PS'],
 ['cond-mat.str-el', 'cond-mat.stat-mech'],
 ['math.RA'],
 ['cond-mat.mes-hall'],
 ['math.CA', 'math.PR'],
 ['hep-ph'],
 ['cond-mat.str-el'],
 ['hep-ph'],
 ['hep-ph'],
 ['physics.optics', 'physics.comp-ph'],
 ['q-bio.PE', 'q-bio.CB', 'quant-ph'],
 ['physics.optics', 'physics.comp-ph'],
 ['q-bio.QM', 'q-bio.MN'],
 ['physics.optics', 'physics.comp-ph'],
 ['physics.optics', 'physics.comp-ph'],
 ['hep-ph', 'hep-lat', 'nucl-th'],
 ['math.OA', 'math.FA'],
 ['math.QA', 'math-ph', 'math.MP'],
 ['physics.gen-ph', 'quant-ph'],
 ['cond-mat.stat-mech', 'cond-mat.

In [14]:
unique_categories = set([i for l in categories for i in  l])
len(unique_categories)
unique_categories

{'acc-phys',
 'adap-org',
 'alg-geom',
 'ao-sci',
 'astro-ph',
 'astro-ph.CO',
 'astro-ph.EP',
 'astro-ph.GA',
 'astro-ph.HE',
 'astro-ph.IM',
 'astro-ph.SR',
 'atom-ph',
 'bayes-an',
 'chao-dyn',
 'chem-ph',
 'cmp-lg',
 'comp-gas',
 'cond-mat',
 'cond-mat.dis-nn',
 'cond-mat.mes-hall',
 'cond-mat.mtrl-sci',
 'cond-mat.other',
 'cond-mat.quant-gas',
 'cond-mat.soft',
 'cond-mat.stat-mech',
 'cond-mat.str-el',
 'cond-mat.supr-con',
 'cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OH',
 'cs.OS',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'dg-ga',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'funct-an',
 'gr-qc',
 'hep-ex',
 'hep-lat',
 'hep-ph',
 'hep-th',
 'm

* 得到2019年以后的所有种类的论文  
    我们的任务要求对于2019年以后的paper进行分析，所以首先对于时间特征进行预处理，从而得到2019年以后的所有种类的论文：

In [ ]:
data["year"] = pd.to_datetime(data["update_date"]).dt.year #将update_date从例如2019-02-20的str变为datetime格式，并提取处year
del data["update_date"] #删除 update_date特征，其使命已完成
data = data[data["year"] >= 2019] #找出 year 中2019年以后的数据，并将其他数据删除
# data.groupby(['categories','year']) #以 categories 进行排序，如果同一个categories 相同则使用 year 特征进行排序
data.reset_index(drop=True, inplace=True) #重新编号
data #查看结果

* 挑选出计算机领域的所有文章


In [ ]:
#爬取所有的类别
website_url = requests.get('https://arxiv.org/category_taxonomy').text #获取网页的文本数据
soup = BeautifulSoup(website_url,'lxml') #爬取数据，这里使用lxml的解析器，加速
root = soup.find('div',{'id':'category_taxonomy_list'}) #找出 BeautifulSoup 对应的标签入口
tags = root.find_all(["h2","h3","h4","p"], recursive=True) #读取 tags

#初始化 str 和 list 变量
level_1_name = ""
level_2_name = ""
level_2_code = ""
level_1_names = []
level_2_codes = []
level_2_names = []
level_3_codes = []
level_3_names = []
level_3_notes = []

#进行
for t in tags:
    if t.name == "h2":
        level_1_name = t.text    
        level_2_code = t.text
        level_2_name = t.text
    elif t.name == "h3":
        raw = t.text
        level_2_code = re.sub(r"(.*)\((.*)\)",r"\2",raw) #正则表达式：模式字符串：(.*)\((.*)\)；被替换字符串"\2"；被处理字符串：raw
        level_2_name = re.sub(r"(.*)\((.*)\)",r"\1",raw)
    elif t.name == "h4":
        raw = t.text
        level_3_code = re.sub(r"(.*) \((.*)\)",r"\1",raw)
        level_3_name = re.sub(r"(.*) \((.*)\)",r"\2",raw)
    elif t.name == "p":
        notes = t.text
        level_1_names.append(level_1_name)
        level_2_names.append(level_2_name)
        level_2_codes.append(level_2_code)
        level_3_names.append(level_3_name)
        level_3_codes.append(level_3_code)
        level_3_notes.append(notes)

#根据以上信息生成dataframe格式的数据
df_taxonomy = pd.DataFrame({
    'group_name' : level_1_names,
    'archive_name' : level_2_names,
    'archive_id' : level_2_codes,
    'category_name' : level_3_names,
    'categories' : level_3_codes,
    'category_description': level_3_notes
    
})

#按照 "group_name" 进行分组，在组内使用 "archive_name" 进行排序
df_taxonomy.groupby(["group_name","archive_name"])
df_taxonomy